### This notebook illustrates an example of viewing information from logged experiments on mlflow. It also shows how to load a model from and perform prediction.

## Imports

In [ ]:
import sys
sys.path.append("..")

import mlflow
import pandas as pd

import config.config as config
import tasks.predict as predict

## Define functions to get model parameters and logged metrics

In [ ]:
def get_parameters_list(data):
    params  = data['params']
    return [params['model'], params['dataset']]


def get_metrics_list(data):
    if ('metrics' in data.keys()) and data['metrics']:
        metrics = data['metrics']
        return [metrics['accuracy'], metrics['f1_score']]
    else:
        return ['-', '-']

## List all existing experiments

In [ ]:
all_experiments = mlflow.list_experiments()
exp_data = []
for exp in all_experiments:
    row = [exp.name, exp.artifact_location, exp.lifecycle_stage]
    exp_data.append(row)
exp_frame = pd.DataFrame(exp_data, columns = ['Name', 'Artifacts', 'Status'])
print(exp_frame)

## For model training experiment - display last 100 runs with a subset of parameters and metrics

In [ ]:
exp = mlflow.get_experiment_by_name('Default')
runs = mlflow.list_run_infos(exp.experiment_id, max_results=100)
data = []
for r in runs:
    run_data = mlflow.get_run(r.run_id).to_dictionary()['data']
    row = get_parameters_list(run_data) + get_metrics_list(run_data)
    data.append(row)
frame = pd.DataFrame(data, columns=['Model', 'Dataset', 'Accuracy', 'F1 Score'])
print('------------------- Model Training Runs ----------------------')
print(frame)

## Show training metrics for the latest model

In [ ]:
from mlflow.tracking import MlflowClient
import matplotlib.pyplot as plt
import numpy as np

client = MlflowClient()

runs = mlflow.list_run_infos(exp.experiment_id, max_results=100)
current = runs[0]
losses = client.get_metric_history(current.run_id, 'epoch_loss')
y = [l.value for l in losses]

fig = plt.figure()
ax = plt.axes()
x = list(range(len(losses)))

plt.plot(x, y, '-', linewidth=3)
plt.xlabel('epoch')
plt.ylabel('loss')

plt.grid()
plt.show()

## Load environment variable overrides and configuration

In [ ]:
%load_ext dotenv
%dotenv
cfg = config.read_config('../config/gemini.cfg')
cfg.model_path = './model.pt'
cfg.result_output = './temp_result.csv'

## Run inference and predict

In [ ]:
predict.main(cfg)